In [2]:
class stock_data_request:
    import pandas as pd
    import requests as req
    import numpy as np
    
    def __init__(self,end_point=None, function=None, symbol=None, outputsize=None, datatype=None, apikey=None):
        self.end_point=end_point
        self.function=function
        self.symbol=symbol
        self.outputsize=outputsize
        self.datatype=datatype
        self.apikey=apikey
        
        if self.end_point is None:
            self.endpoint = 'https://www.alphavantage.co/query'
        if self.function is None:
            self.function = 'TIME_SERIES_DAILY'
        if self.symbol is None:
            self.symbol ='sp'
        if self.outputsize is None:
            self.outputsize = 'compact'
        if self.datatype is None:
            self.datatype ='json'
        if self.apikey is None:
            self.apikey='YR6FVM72V0C8DPQL'
    
    def pull(self): 
    
        parameters = {
            "function": self.function,
            "symbol": self.symbol,
            "outputsize": self.outputsize,
            "datatype": self.datatype,
            "apikey": self.apikey,
        }

        r = req.get(url=self.endpoint, params=parameters)

        data = r.json()

        return data
    
class stock_data_frame(stock_data_request):
    
    %matplotlib inline
    import matplotlib.pyplot as plt
    import statsmodels.api as sm
        
    def __init__(self, symbol=None, data=None, end_point=None, function=None, outputsize=None, datatype=None, apikey=None):
        stock_data_request.__init__(self,symbol=None, end_point=None, function=None, outputsize=None, datatype=None, apikey=None)
        self.symbol=symbol
        self.data=data
        if self.data is None:
            self.data = stock_data_request(symbol=self.symbol).pull()

    def construct(self):

        data = self.data["Time Series (Daily)"]

        opens, highs, lows, closes, volumes, dates = [], [], [], [], [], []
        for major_key, sub_dict in data.items():
            dates.append(major_key)
            opens.append(sub_dict["1. open"])
            highs.append(sub_dict["2. high"])
            lows.append(sub_dict["3. low"])
            closes.append(sub_dict["4. close"])                
            volumes.append(sub_dict["5. volume"])

        df = pd.DataFrame(

            {
                "date": dates,
                "open": opens,
                "high": highs,
                "low": lows,
                "close": closes,                    
                "volume": volumes,
            }

        )
        df["date"]= pd.to_datetime(df['date'])
        df["open"] = df["open"].astype(float)
        df["high"] = df["high"].astype(float)
        df["low"] = df["low"].astype(float)
        df["close"] = df["close"].astype(float)
        df["volume"] = df["volume"].astype(int)

        return df

    def info(self):

        symbol = self.data['Meta Data']['2. Symbol']
        last_refresh = self.data['Meta Data']['3. Last Refreshed']
        time_zone = self.data['Meta Data']['5. Time Zone']

        meta_list = [symbol, last_refresh, time_zone]

        return meta_list
    
    # Takes dataframe as optional argument, otherwise defaults to S&P 500 dataframe
    # returns regression summary
def regression(ind, dep=None, plot=None):
    if dep is None:
        dep = stock_data_frame('sp').construct()
            
    r_ind = [((x-y)/x) for x, y in zip(ind['close'], ind['close'][1:])]

    r_dep = [((x-y)/x) for x, y in zip(dep['close'], dep['close'][1:])]
        
    x = np.array(r_ind)
    y = np.array(r_dep)
    x_ = sm.add_constant(x.reshape(-1,1))

    model = sm.OLS(y, x_)
    result = model.fit()
        
    rsq = result.rsquared
    coefs = result.params
        
    fig = plt.figure 
        
    plt.scatter(x, y)
    plt.title("Linear Regression: ",loc='center')
    plt.table([['',''],['',''],["R-Squared: ", rsq], ["B0: ", coefs[0]],["X1: ", coefs[1]]])
        
    return plt.show()
        
        
        
        
        
        
        
    


    

In [4]:
dep = stock_data_frame('aapl').construct()


NameError: name 'req' is not defined

In [ ]:
regression(dep)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     21.05
Date:                Sun, 20 Oct 2019   Prob (F-statistic):           1.34e-05
Time:                        00:20:59   Log-Likelihood:                 279.84
No. Observations:                  99   AIC:                            -555.7
Df Residuals:                      97   BIC:                            -550.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0018      0.001      1.236      0.220      -0.001       0.005
x1             0.4422      0.096      4.588      0.000       0.251       0.634
==============================================================================
Omnibus:                        5.538   Durbin-Watson:                   2.289
Prob(Omnibus):                  0.063   Jarque-Bera (JB):                5.102
Skew:                          -0.419   Prob(JB):                       0.0780
Kurtosis:                       3.731   Cond. No.                         66.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [126]:

rsq = result.rsquared
coefs = result.params

rsq, coefs

(0.178281259599406, array([0.0018142 , 0.44222314]))